In [3]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch

In [5]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1-input x1)
   [0.55, 0.87, 0.66], # journey  (x^2-input x2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [6]:
#input query-word "jouney"
x_2=inputs[1]
#for all matrices(key,query,value)-dimension should match with input matrix(3x2 here)

d_in=inputs.shape[1]#3
d_out=2

#in gt models,input and output dimensions are same

In [7]:

torch.manual_seed(123)
W_query=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
#matrix initialization with random values for query weight,key weight and value weight matrices with dimension of 3x2

In [8]:
#calculating query vector,key vector,value vector for input("journey")

query_2=x_2@W_query
key_2=x_2@W_key
value_2=x_2@W_value
print(query_2)

tensor([0.4306, 1.4551])


In [9]:
#now obtain a matrix that contains keys,queries,values for all the inputs

keys=inputs@W_key
queries=inputs@W_query
values=inputs@W_value

print("Keys shape: ",keys.shape)
print("Values shape: ",values.shape)
print("queries shape: ",queries.shape)

#here all the tokens are projected down from 3d space(token embeddings) to 2d space(query,key,value)

Keys shape:  torch.Size([6, 2])
Values shape:  torch.Size([6, 2])
queries shape:  torch.Size([6, 2])


In [10]:
#finding attention score  for second query("jouney")
keys_2=keys[1] 
attn_score_2=torch.dot(keys_2)
print(attn_score_2)

TypeError: dot() missing 1 required positional arguments: "tensor"

In [ ]:
#finding attention score  for second query("journey")  with respect to all other keys
attn_scores_2=query_2@keys.T#query_2 represent the value from queries matrix for word "journey" (2nd row)
print(attn_scores_2)
print(attn_scores_2.shape)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])
torch.Size([6])


In [ ]:
#finding attention score  for all queries  with respect to all other keys
attn_scores=queries@keys.T
print(attn_scores)

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


In [ ]:
d_k=keys.shape[-1]#dimension of keys matrix(here 2d)
#attention weight for 2nd token("journey")
attn_weights_2=torch.softmax(attn_scores_2/d_k**0.5,dim=-1)#d_k**0.5=sqrt(2) and dim=-1 means summing up all the columnvalues in row for softmax denominator in each value
print(attn_weights_2)


tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [ ]:
#reasons for applying sqrt(dimension) - stability in learning
tensor=torch.tensor([0.1,-0.2,0.3,-0.2,0.5])

softmax_result=torch.softmax(tensor,dim=-1)
print("Softmax with scaling: ",softmax_result)

tensor_scaled=tensor*8
softmax_result_scaled=torch.softmax(tensor_scaled,dim=-1)
print("Softmax after scaling(tensor * 8): ",softmax_result_scaled)

#here in result,before scaling the values in results are distributed correctly
#but after scaling the tensor,the probability gets scattered and highest value in tensor captures 80%(0.5 in input tensor captures 80% in scaled_tensor).
#it causes instability in learning

Softmax with scaling:  tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
Softmax after scaling(tensor * 8):  tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [ ]:
#calculating context vector for word journey with values
context_vec_2=attn_weights_2@values
print(context_vec_2)

tensor([0.3061, 0.8210])


In [ ]:
#Implementing a self attention python class
import torch.nn as nn
#nn is a fundamental building block of pytorch ,which provides necssary services for model creation and management
class SelfAttentionV1(nn.Module):

    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_keys=nn.Parameter(torch.rand(d_in,d_out))
        self.W_values=nn.Parameter(torch.rand(d_in,d_out))
        self.W_queries=nn.Parameter(torch.rand(d_in,d_out))

    
    def forward(self,x):#x represenst input embedding vectors

        keys=x@self.W_keys
        queries=x@self.W_queries
        values=x@self.W_values

        attn_scores=queries@keys.T
        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
        context_vec=attn_weights@values
        return context_vec

In [ ]:
sa_v1=SelfAttentionV1(d_in,d_out)
print(sa_v1(inputs))

tensor([[0.8413, 0.8945],
        [0.8719, 0.9321],
        [0.8713, 0.9312],
        [0.8240, 0.8726],
        [0.8292, 0.8783],
        [0.8361, 0.8867]], grad_fn=<MmBackward0>)


In [ ]:
#Implementing a self attention python class
import torch.nn as nn
#nn is a fundamental building block of pytorch ,which provides necssary services for model creation and management
#Linear is used because it effectively performs matrix multiplicationa when bias units are disabled and it has an optimized weight initialization scheme while initializing matrices with random values contributing to more stable and effective model training
class SelfAttentionV2(nn.Module):

    def __init__(self,d_in,d_out,qkv_bias=False):
        super().__init__()
        self.W_keys=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_values=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_queries=nn.Linear(d_in,d_out,bias=qkv_bias)

    
    def forward(self,x):#x represenst input embedding vectors

        keys=self.W_keys(x)
        queries=self.W_queries(x)
        values=self.W_values(x)

        attn_scores=queries@keys.T
        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
        context_vec=attn_weights@values
        return context_vec

In [ ]:
sa_v2=SelfAttentionV2(d_in,d_out)
print(sa_v2(inputs))
#output from self attention version 1 is different from version 2 because the weight initialization is more sophisticated for linear than parameter

tensor([[ 0.7148, -0.0405],
        [ 0.7132, -0.0415],
        [ 0.7132, -0.0413],
        [ 0.7136, -0.0388],
        [ 0.7140, -0.0348],
        [ 0.7133, -0.0415]], grad_fn=<MmBackward0>)


In [ ]:
#casual attention -masking out future tokens
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1-input x1)
   [0.55, 0.87, 0.66], # journey  (x^2-input x2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
queries=sa_v2.W_queries(inputs)
keys=sa_v2.W_keys(inputs)
attn_scores=queries@keys.T
attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
print(attn_weights)

tensor([[0.1757, 0.1646, 0.1655, 0.1603, 0.1812, 0.1527],
        [0.1767, 0.1622, 0.1634, 0.1602, 0.1884, 0.1492],
        [0.1763, 0.1623, 0.1635, 0.1605, 0.1876, 0.1498],
        [0.1727, 0.1639, 0.1646, 0.1629, 0.1799, 0.1559],
        [0.1667, 0.1660, 0.1661, 0.1669, 0.1682, 0.1661],
        [0.1767, 0.1623, 0.1635, 0.1601, 0.1881, 0.1492]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
#use pytorch tril(triangle lower) function to create a mask where the values above diagonal is zero
context_length=attn_scores.shape[0]
mask_simple=torch.tril(torch.ones(context_length,context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [ ]:
#masking the above diagonal in attention weights
masked_attn_scores=attn_weights*mask_simple
print(masked_attn_scores)

tensor([[0.1757, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1767, 0.1622, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1763, 0.1623, 0.1635, 0.0000, 0.0000, 0.0000],
        [0.1727, 0.1639, 0.1646, 0.1629, 0.0000, 0.0000],
        [0.1667, 0.1660, 0.1661, 0.1669, 0.1682, 0.0000],
        [0.1767, 0.1623, 0.1635, 0.1601, 0.1881, 0.1492]],
       grad_fn=<MulBackward0>)


In [ ]:
#again normalizing the attention weights to sum up to 1 in each row in masked attention scores
row_sums=masked_attn_scores.sum(dim=1,keepdims=True)
#calculating sum of all values in each row
masked_simple_norm=masked_attn_scores/row_sums
#dividing each value in a row with that row sum
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5214, 0.4786, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3512, 0.3233, 0.3256, 0.0000, 0.0000, 0.0000],
        [0.2600, 0.2468, 0.2479, 0.2453, 0.0000, 0.0000],
        [0.1998, 0.1991, 0.1992, 0.2002, 0.2017, 0.0000],
        [0.1767, 0.1623, 0.1635, 0.1601, 0.1881, 0.1492]],
       grad_fn=<DivBackward0>)


In [ ]:
#more efficient way to mask is to mask the attention scores
print(attn_scores)

tensor([[ 0.0957,  0.0035,  0.0107, -0.0339,  0.1391, -0.1026],
        [ 0.0618, -0.0595, -0.0491, -0.0770,  0.1525, -0.1779],
        [ 0.0587, -0.0583, -0.0483, -0.0747,  0.1466, -0.1722],
        [ 0.0326, -0.0414, -0.0350, -0.0496,  0.0906, -0.1120],
        [-0.0146, -0.0200, -0.0193, -0.0122, -0.0014, -0.0196],
        [ 0.0655, -0.0549, -0.0446, -0.0743,  0.1533, -0.1737]],
       grad_fn=<MmBackward0>)


In [ ]:
#masking upper diagonal of attention scores with -infinity
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
masked=attn_scores.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[ 0.0957,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0618, -0.0595,    -inf,    -inf,    -inf,    -inf],
        [ 0.0587, -0.0583, -0.0483,    -inf,    -inf,    -inf],
        [ 0.0326, -0.0414, -0.0350, -0.0496,    -inf,    -inf],
        [-0.0146, -0.0200, -0.0193, -0.0122, -0.0014,    -inf],
        [ 0.0655, -0.0549, -0.0446, -0.0743,  0.1533, -0.1737]],
       grad_fn=<MaskedFillBackward0>)


In [ ]:
#normaliizng using softmax
attn_weights=torch.softmax(masked/keys.shape[-1]**0.5,dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5214, 0.4786, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3512, 0.3233, 0.3256, 0.0000, 0.0000, 0.0000],
        [0.2600, 0.2468, 0.2479, 0.2453, 0.0000, 0.0000],
        [0.1998, 0.1991, 0.1992, 0.2002, 0.2017, 0.0000],
        [0.1767, 0.1623, 0.1635, 0.1601, 0.1881, 0.1492]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
#dropout layer to mask out 50% which means masking out half of attention weights
dropout=torch.nn.Dropout(0.5)
example=torch.ones(6,6)
print(example)
#50% of weights in each row is switched off 
#50% means half of elements in matrix are reduced to 0. to compensate this,other values are sacled up to 2(1/0.5=2)
print(dropout(example))

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0., 2., 2., 0., 2., 2.],
        [2., 2., 2., 0., 0., 2.],
        [2., 0., 2., 0., 0., 0.],
        [0., 2., 2., 2., 2., 0.],
        [2., 2., 2., 0., 0., 2.],
        [2., 2., 0., 2., 2., 0.]])


In [ ]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7023, 0.6465, 0.6512, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4935, 0.4958, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3982, 0.0000, 0.4004, 0.0000, 0.0000],
        [0.0000, 0.3246, 0.3270, 0.3202, 0.3761, 0.0000]],
       grad_fn=<MulBackward0>)


In [ ]:
#creating a batch of inputs
batch=torch.stack((inputs,inputs),dim=0)
print(batch.shape)
#th result is a 3d tensor consisting of 2 input texts with 6 tokens each where each token is a 3-dimensional embedding vector

torch.Size([2, 6, 3])


In [ ]:
#casual attention class-handling batches of inputs
class CasualAttention(nn.Module):

    def __init__(self,d_in,d_out,context_length,dropout,qkv_bias=False):
        super().__init__()
        self.d_out=d_out
        self.W_keys=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_values=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_queries=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout=nn.Dropout(dropout)
        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        self.register_buffer("mask", mask)
        #buffer are automatically moved to appropriate device(GPU or CPU) along with model which will help in training LLM
    
    def forward(self,x):#x represenst input embedding vectors
        b,num_tokens,d_in=x.shape#b-batches,num_tokens-no.of.tokens in each input,d_in-dimensions for each token
        keys=self.W_keys(x)
        queries=self.W_queries(x)
        values=self.W_values(x)

        attn_scores=queries@keys.transpose(1,2)#here in input dimension is (2,6,3) we need only want that (6,3) so use indexing to achive that(1,2).6-1st ind,3-2nd ind
        #filling upper diagonal with mask of negative infinity
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf)#here :num_tokens is used to account for cases where number of tokens in the batch is smaller than supported context_size
        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
        #adding dropout layer
        attn_weights=self.dropout(attn_weights)
        context_vecs=attn_weights@values
        return context_vecs

In [ ]:
torch.manual_seed(123)
context_length=batch.shape[1]#batch shape-(2,6,3) here 6 is the context length
ca=CasualAttention(d_in,d_out,context_length,0.0)
context_vectors=ca(batch)
print("context vector shape: ",context_vectors.shape)
print(context_vectors)
#here result is a 3d vector with 2 batches of inputs with each having 6 tokens with each token having a 2d context vector

context vector shape:  torch.Size([2, 6, 2])
tensor([[[-0.5740,  0.2727],
         [-0.7249,  0.1854],
         [-0.7714,  0.1586],
         [-0.6997,  0.1210],
         [-0.6560,  0.1309],
         [-0.6445,  0.1027]],

        [[-0.5740,  0.2727],
         [-0.7249,  0.1854],
         [-0.7714,  0.1586],
         [-0.6997,  0.1210],
         [-0.6560,  0.1309],
         [-0.6445,  0.1027]]], grad_fn=<UnsafeViewBackward0>)


In [ ]:
#multi head attention- multiple simple attention for single input 
#each input embedding vector will have multiple instances of key,value,query weights matrices and multiple queries,keys,values matrices and multiple context vectors and those context vectors are conactenation at last

class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super().__init__()
        #creating multiple instances of casual attention
        self.heads=nn.ModuleList(
            [CasualAttention(d_in,d_out,context_length,dropout,qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)
        #concatenating context vectors from each head along columns

In [ ]:
torch.manual_seed(123)
context_length=batch.shape[1]
d_in,d_out=3,2
mha=MultiHeadAttentionWrapper(d_in,d_out,context_length,0.0,num_heads=2)
mha_context_vecs=mha(batch)
print(mha_context_vecs)
print("shape: ",mha_context_vecs.shape)
#the result is a 3d vector with batch of size 2 with each input having 6 tokens and each token having a context vector of 4d context vector(4d beacuse 2d context vector from 2 heads added )

tensor([[[-0.5740,  0.2727, -0.3132, -0.2272],
         [-0.7249,  0.1854, -0.2330,  0.0261],
         [-0.7714,  0.1586, -0.2079,  0.1126],
         [-0.6997,  0.1210, -0.1678,  0.1287],
         [-0.6560,  0.1309, -0.1702,  0.1722],
         [-0.6445,  0.1027, -0.1425,  0.1594]],

        [[-0.5740,  0.2727, -0.3132, -0.2272],
         [-0.7249,  0.1854, -0.2330,  0.0261],
         [-0.7714,  0.1586, -0.2079,  0.1126],
         [-0.6997,  0.1210, -0.1678,  0.1287],
         [-0.6560,  0.1309, -0.1702,  0.1722],
         [-0.6445,  0.1027, -0.1425,  0.1594]]], grad_fn=<CatBackward0>)
shape:  torch.Size([2, 6, 4])


In [16]:
#improving multi head attention forward method by processing multiple heads sequentially
#here instaed of making multiple weight matrices for key,value and query we take one large matrix (for eg: if there are 2 heads with 2 separate weight matrix ,the output will be 2 different matrixes with dimension of (3x2) then add it along columns will result in (3x4) matrix 
# but here we are taking one large weight matrix with dimension of (3x4) and find query,key,value matrices and split them with num_heads(2) ,result in two 3x2 matrices)
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,num_heads,dropout,qkv_bias=False):
        super().__init__()
        assert (d_out%num_heads==0),"d_out must be divisible by num_heads"

        self.d_out=d_out
        self.num_heads=num_heads
        self.head_dim = d_out//num_heads #finding dimension of each head
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.out_proj=nn.Linear(d_out,d_out)
        self.dropout=nn.Dropout(dropout)
        self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):
        b,num_tokens,d_in=x.shape

        #shape for keys,queries,values matrix=(b,num_tokens,d_out)
        keys=self.W_key(x)
        queries=self.W_query(x)
        values=self.W_value(x)

        #split the larger matrices(keys,queries,values) according to num of heads
        #unroll last dimension to split the matrices according to no of heads:(b,num_tokens,d_out)->(b,num_tokens,num_heads,head_dim)
        keys=keys.view(b,num_tokens,self.num_heads,self.head_dim)
        queries=queries.view(b,num_tokens,self.num_heads,self.head_dim) 
        values=values.view(b,num_tokens,self.num_heads,self.head_dim) 

        #grouping according to num of heads by transposing
        #(b,num_tokens,num_heads,head_dim)->(b,num_heads,num_tokens,head_dim)
        keys=keys.transpose(1,2)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)

        #computing attention scores
        attn_scores=queries@keys.transpose(2,3)
        #here, each row i in each head represents the attention score of ith token with respect to all tokens in that head

        #implementing mask for upper diagonal
        mask_bool=self.mask.bool()[:num_tokens,:num_tokens]

        #masking the attention scores
        attn_scores.masked_fill_(mask_bool,-torch.inf)

        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)#keys.shape[1] refers to head_dim
        attn_weights=self.dropout(attn_weights)

        #context vector: shape->(b,num_tokens,num_heads,head_dim)
        context_vec=(attn_weights@values).transpose(1,2)

        #combine heads,where d_out=num_heads*head_dim
        context_vec=context_vec.contiguous().view(b,num_tokens,self.d_out)#values are stored sometimes non-contiguously in memory.if dimesnion changed on non-contiguously,it will lead to error.This method(contiguous) makes a new tensor with the same values but stored in a clean, contiguous memory block.
        context_vec=self.out_proj(context_vec)#optional projection layer

        return context_vec






In [ ]:
torch.manual_seed(123)

# Define the tensor with 3 rows and 6 columns
inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],  # Row 1
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],  # Row 2
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]  # Row 3
)

batch=torch.stack((inputs,inputs),dim=0)
print(batch.shape)

batch_size,context_length,d_in=batch.shape
d_out=6
mha=MultiHeadAttention(d_in,d_out,context_length,2,0.0)
context_vecs=mha(batch)
print(context_vecs)
print("context vector shape: ",context_vecs.shape)
#result is a 3d vector for 2 inputs in batch with each input having 3 tokens and each token with 6d context vectors(6d because of multiple heads,if divide it by num_heads(6/2=3),it will split into two 3d vectors each vector is for separate heads)

torch.Size([2, 3, 6])
tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1125, -0.0561,  0.0454, -0.0234, -0.3247, -0.3030],
         [ 0.1194, -0.0501,  0.0347, -0.0637, -0.2797, -0.2612]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1125, -0.0561,  0.0454, -0.0234, -0.3247, -0.3030],
         [ 0.1194, -0.0501,  0.0347, -0.0637, -0.2797, -0.2612]]],
       grad_fn=<ViewBackward0>)
context vector shape:  torch.Size([2, 3, 6])
